# **LDA(Latent Dirichlet Allocation)**

# **Settings**

In [ ]:
! pip install koreanize_matplotlib

In [ ]:
! pip install -U pyLDAvis

In [ ]:
! pip install konlpy

In [2]:
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/메타버스 아카데미/Data/')
import warnings
warnings.filterwarnings('ignore')                       # warning 출력 false

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import koreanize_matplotlib

## ***! Note LDA***
* 토픽 모델링(Topic Learning)의 한 종류
* 토픽은 각각 단어 분포를 가지고 있다고 가정
* 과정
    1. 적절한 토픽의 개수를 지정해주는 게 좋다(5개 정도)
    2. 토픽의 혼합 비율을 설정(무작위)
    3. 모든 단어를 특정 토픽에 할당
    4. 반복
        1. 현재 문서 d의 모든 단어 w, 해당되는 토픽에 재할당
        2. 현재 문서 d에 토픽의 비율 업데이트
        3. 모든 문서에서 토픽의 비율 업데이트
    5. 최종적으로 할당된 비율을 통해 문서의 주체를 추정

    

# **1. 데이터 불러오기**

In [6]:
data = pd.read_csv('./appreply.csv',index_col=0)
data = data.iloc[3:,:].reset_index(drop=True)
data

,text,score
0,"배달의민족 주문시 리뷰를 자주 참고하는 편입니다. 한가지 건의사항이 있다면 최신순,...",4
1,내가 주문했던 과거목록에서도 검색기능이 있었으면 좋겠어요.. 분명 이 가게에 시킨 ...,5
2,"검색 화면에서 전체/배달/포장 탭 중 배달 탭을 스크롤 내리면서 볼 때, 아래로 스...",1
3,배달팁 낮은 순으로 정렬하면 0~4000원 이런식으로 된 가게가 가장 위로 올라옵니...,2
4,최근 업데이트가 안드로이드5사양 정도에서는 안되는것 같습니다.. 배민 어플 실행시 ...,3
...,...,...
995,갑자기 로그아웃 되더니 비밀번호 변경 실패 메세지가 계속 뜨네요. 휴대폰 번호로 인...,1
996,기사님이 상품 픽업을 하셨는지 표시되면 더 좋을 것 같습니다. 가게에서 조리가 늦게...,3
997,요즘 요기요 보다 배민을 많이 쓰는 사람입니다 전화보다 앱을 써서 좀더 간편하고 다...,3
998,취소 됐으면 적어도 전화 주는 제도는 있어야하는거 아닌가요? 주문해놓고 다른거 하는...,1


# **2. 데이터 전처리**

## **1) 형태소 분석**

In [15]:
from konlpy.tag import Okt
from tqdm import tqdm
import re

def tokenize_text(text):
    text = re.sub('[^ㄱ-ㅣ가-힣\s]','',text)
    okt = Okt()
    okt_morphs = okt.pos(text)

    result = []
    for word, pos in okt_morphs:
        if pos in ['Adjective','Verb','Noun']:
            result.append(word)

    return ' '.join(result)

In [18]:
token_list = []
for text in tqdm(data['text']):
    token = tokenize_text(text)
    token_list.append(token)

print(token_list)

100%|██████████| 1000/1000 [00:14<00:00, 69.40it/s]

['배달 민족 주문 시 리뷰 자주 참고 하는 편입 니 한가지 건의 사항 있다면 최신 점순 뿐 아니라 제 주문 하고 자하 메뉴 특정해서 그 메뉴 리뷰 확인 할 수 있는 기능 있으면 좋을 것 같습니다 메뉴 검색 기능 리뷰 특정 메뉴 검색 기능 필요합니다 주문 수가 많지 않은 메뉴 리뷰 보기 위해 드 래그 하느라 시간 소요 되는 비 효율 발생 합니다 긍정 검토 해 주심 좋을 것 같습니다', '내 주문 했던 과거 목록 검색 기능 있었으면 좋겠어요 분명 이 가게 시킨 기억 있는데 뭘 시켰고 맛있게 먹은게 맞는지 알고싶은데 찾기 어렵네요 먹던 메뉴 검색 가능했으면 좋겠어요 예 들면 곱창 치면 과거 시킨 곱창 목록 뜨거나 가게 리뷰 창 리뷰 보기 가 있어도 좋을거 같아요', '검색 화면 전체 배달 포장 탭 중 배달 탭 스 크롤 내리면서 볼 때 아래 스 크롤 하는데 자꾸만 왼쪽 오른쪽 넘어가서 전체 탭 포장 탭 보게 되는데 정말 불편합니다 검색 후 배달 탭 포장 탭 마트 탭 뭐 하나 선택 했으면 좌우 넘기는 건 안 하시면 안되나요 가끔 그러는 것 아니고 스 크롤 한 번 내릴 때 왼쪽 넘어가서 전체 탭 가게 되니까 불편하고 답답해서 씁니다', '배달 팁 낮은 순 정렬 하면 원 식 된 가게 가장 위로 올라옵니다 지역 따라 추가 배달 료 있다고 별도 체크 하게 되어있어서 배달 팁 원 넘습니다 만원 이상 시키면 배달 팁 원 데 장난 하는 것 아니고 하나 하나 눌러가면서 가격 대별 금액 체크 배달 팁 별도 받는지도 확인 해야 합니다', '최근 업데이트 안드로이드 사양 정도 안되는것 같습니다 배민 어플 실행 시 업데이트 하라 나오고 업데이트 누르면 업데이트 진행 안되고 열기 나오고 열기 누르면 업데이트 나오고 무한 반복 삭제 재 설치 환경설정 등 해보아도 같은 증상 요 다른 같은 증상 보이는 글 보니 이번 업데이트 관련 파일 확인 필요해 보입니다 참고 사양 높은 폰 되는거 보니 핸드폰 사양 문제 이번 업데이트 있어 보입니다', '찜 매장 구분 좀 해주세요 대체 언제 이면 개선 될까 요

## **2) 필요없는 문장 제거**

In [19]:
# 짧은 문장 찾기
remove_list = []
for index in range(len(token_list)):
    token = token_list[index]
    if len(set(token.split())) < 3:
        remove_list.append(token)
print(f'remove list : {len(remove_list)}')

# 짧은 문장 제거
print(f'Before: {len(token_list)}')
for token in remove_list:
    token_list.remove(token)
print(f'After : {len(token_list)}')

remove list : 0
Before: 1000
After : 1000


# **3. LDA**

In [20]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [22]:
# LDA는 Count 기반의 vectorizer만 사용해야 한다.
# max_df=p, 전체 단어의 등장 비율이 p이상인 것만 사용하겠다
# min_df=n, 이 단어가 적어도 n개 이상의 document에 등장한 것만 사용하겠다
# ngram_range=(), 단어의 조합을 설정하겠다.

count_vectorizer = CountVectorizer(max_df=0.1, min_df=2, max_features=1000, ngram_range=(1,2))
feat_vec = count_vectorizer.fit_transform(token_list)
feat_vec.shape

(1000, 1000)

In [23]:
lda = LatentDirichletAllocation(n_components=5)         # topic의 개수
lda.fit(feat_vec)

LatentDirichletAllocation(n_components=5)

In [25]:
feature_names = count_vectorizer.get_feature_names_out()
print(feature_names)

['가게 검색' '가게 메뉴' '가게 목록' '가게 배달' '가게 주문' '가격' '가기' '가까운' '가끔' '가능' '가능하게'
 '가능한' '가맹' '가서' '가요' '가입' '가장' '가족' '가지' '갈수록' '감사합니다' '갑자기' '강제' '같고'
 '같네요' '같습니다' '같아' '같아서' '같아요' '같은' '같은 경우' '같은거' '같은데' '같음' '개발' '개발자'
 '개인' '개편' '갤럭시' '거기' '거나' '거리' '거의' '건가' '건물' '건의' '건의 사항' '건지' '걸리고'
 '검색 기능' '겁니다' '결과' '결재' '결제 수단' '경우' '경험' '계속' '계정' '고객' '고객 센터' '고려'
 '고민' '고쳐주세요' '공지' '과정' '관련' '관리' '광고' '굉장히' '구매' '구분' '구성' '굳이' '그거' '그것'
 '그게' '그냥' '그대로' '그런' '그럴' '그럼' '근처' '글자' '금액' '기간' '기기' '기능 있었으면'
 '기능 있으면' '기능 추가' '기본' '기분' '기사' '기억' '기업' '기요' '기입' '기존' '기준' '까요' '깔고'
 '깜빡' '나서' '나오고' '나오는' '나중' '남깁니다' '낮은' '낮은 정렬' '내고' '내려서' '내용' '넘게'
 '넘어가서' '네이버' '네이버 로그인' '네트워크' '노력' '노출' '높은' '누가' '누구' '누락' '누르고' '누르면'
 '눌러도' '눌러서' '느낌' '는걸' '늦게' '다르게' '다른 배달' '다른 어플' '다만' '다시 설치' '다양한' '다운'
 '다운로드' '다음' '단계' '달라' '답변' '당연히' '대기' '대부분' '대체' '대한' '대한 리뷰' '대해' '대행'
 '댓글' '덕분' '데이터' '도대체' '도움' '도착' '도착 예정' '동네' '동시' '동안' '동의' '돼서' '돼요'
 '됐는데' '됐으면' '되고' '되네요' '되는' '되는 경우' '되는거' '되는건' '되는데' '되는지' '되면'

In [26]:
lda.components_

array([[ 0.20069616,  1.27019881,  0.20000601, ...,  0.38317403,
         0.20070489,  2.92361926],
       [ 7.82525849,  0.20416675,  0.21785109, ...,  0.20048541,
         0.20006582,  0.20792017],
       [ 2.56410936,  5.37817945,  5.63247015, ...,  0.20434224,
         0.20000128,  3.48267333],
       [ 0.20993358,  2.94745102,  2.74953246, ...,  0.20122736,
         0.20005311,  0.2031131 ],
       [ 0.20000242,  0.20000398,  0.20014029, ..., 23.01077096,
         8.1991749 ,  1.18267414]])

In [31]:
def display_topics(model,feature_names, num_top_words):
    for topic_idx, topic in enumerate(model.components_,1):
        print(f'Topic #{topic_idx}.')
        topic_word_idxs = topic.argsort()[::-1]
        top_idxs = topic_word_idxs[:num_top_words]
        topics = [feature_names[idx] for idx in top_idxs]
        print(' '.join(topics))

display_topics(lda, feature_names, num_top_words=20)

Topic #1.
취소 고객 전화 이벤트 했는데 센터 할인 고객 센터 상담 확인 합니다 하고 그냥 서비스 문의 이용 연결 사람 답변 문제
Topic #2.
기사 있습니다 쓰고 좋겠습니다 민족 배달 민족 매장 같은 좋겠어요 추가 이용 정말 하기 경우 선택 같아요 배달 기사 하나 식당 업체
Topic #3.
좋겠어요 금액 시켜 좋겠습니다 있었으면 있으면 별로 사람 만원 천원 추가 최소 수저 표시 이상 가격 사진 선택 이용 낮은
Topic #4.
화면 광고 설정 생각 주소 목록 포장 선택 카테고리 불편해요 진짜 자주 부분 지도 아니라 메인 정렬 그냥 바로 하지
Topic #5.
로그인 오류 인증 계속 요청 문제 해도 카드 안되고 확인 삭제 번호 입력 가입 알림 사항 아이디 설치 요청 사항 회원


In [33]:
import pyLDAvis.lda_model

pyLDAvis.enable_notebook()
vis = pyLDAvis.lda_model.prepare(lda, feat_vec, count_vectorizer)
pyLDAvis.display(vis)

# **4. 문장 매칭하기**

In [36]:
sent_topic = lda.transform(feat_vec)
print(sent_topic.shape)
sent_topic[0]

(1000, 5)


array([0.00521633, 0.66470427, 0.31965217, 0.00524355, 0.00518368])

In [38]:
sent_topic_per_list = []

for n in range(sent_topic.shape[0]):
    topic_idx = sent_topic[n].argmax()
    topic_per = sent_topic[n].max()

    sent_topic_per_list.append([n, topic_idx, topic_per])

topic_per_data = pd.DataFrame(sent_topic_per_list, columns=['no','topic no','prob'])
topic_per_data

,no,topic no,prob
0,0,1,0.664704
1,1,1,0.962958
2,2,3,0.973037
3,3,2,0.481750
4,4,3,0.505315
...,...,...,...
995,995,4,0.885483
996,996,1,0.977762
997,997,0,0.505013
998,998,0,0.543818


In [39]:
doc_topic_data = topic_per_data.join(data)
doc_topic_data

,no,topic no,prob,text,score
0,0,1,0.664704,"배달의민족 주문시 리뷰를 자주 참고하는 편입니다. 한가지 건의사항이 있다면 최신순,...",4
1,1,1,0.962958,내가 주문했던 과거목록에서도 검색기능이 있었으면 좋겠어요.. 분명 이 가게에 시킨 ...,5
2,2,3,0.973037,"검색 화면에서 전체/배달/포장 탭 중 배달 탭을 스크롤 내리면서 볼 때, 아래로 스...",1
3,3,2,0.481750,배달팁 낮은 순으로 정렬하면 0~4000원 이런식으로 된 가게가 가장 위로 올라옵니...,2
4,4,3,0.505315,최근 업데이트가 안드로이드5사양 정도에서는 안되는것 같습니다.. 배민 어플 실행시 ...,3
...,...,...,...,...,...
995,995,4,0.885483,갑자기 로그아웃 되더니 비밀번호 변경 실패 메세지가 계속 뜨네요. 휴대폰 번호로 인...,1
996,996,1,0.977762,기사님이 상품 픽업을 하셨는지 표시되면 더 좋을 것 같습니다. 가게에서 조리가 늦게...,3
997,997,0,0.505013,요즘 요기요 보다 배민을 많이 쓰는 사람입니다 전화보다 앱을 써서 좀더 간편하고 다...,3
998,998,0,0.543818,취소 됐으면 적어도 전화 주는 제도는 있어야하는거 아닌가요? 주문해놓고 다른거 하는...,1


# **결과 파악하기**

In [41]:
# 어떤 topic이 가장 많을까?
doc_topic_data['topic no'].value_counts().sort_index()

topic no
0    243
1    172
2    184
3    192
4    209
Name: count, dtype: int64

In [45]:
# 각 토픽마다 prob이 높은 문장 살펴보기
for topic in range(len(doc_topic_data['topic no'].unique())):
    print(f'Topic # {topic}')
    top_per_topics = doc_topic_data[doc_topic_data['topic no']==topic].sort_values(by='prob',ascending=False)
    for num in range(3):
        print(top_per_topics['text'].iloc[num])


Topic # 0
배민 쭉 써왔고 쓴소리 하나 하려합니다. 중간다리 플랫폼으로서 식당/유저 사이 중재자 역할을 하는 것이라면, 양쪽의 편의성을 제고하는 방향이어야죠. 전체 주문 중 일부 음식만 이상이 있어서 부분환불을 원하는데, 일부취소가 매우 복잡하게 되어 있는 시스템이라(재주문 필요) 식당이 일부취소를 포기하게 만드시네요? 전체 환불해주시겠다는데 그건 말도 안되게 식당 손해이신 것 같아서, 식당과 직접 전화해서 해결하는 중인데요. 앱 개선 제대로 하세요. --- 답변 보고 리뷰 보충하는데요. 저는 지금 고객센터 활용법을 문의/cs 센터에 불만을 제기하는게 아니라(실시간채팅 저도 알아요), 주문 내용의 일부취소가 되지 않고 전체취소 -> 재주문 이라는 방식을 통해 실질적인 일부취소를 우회적으로 만들어내야 하는 현 시스템 개선을 요구하는 것입니다. 우회적으로 일부취소를 해야하는 번거로움 때문에 여러 음식 중 하나의 음식에만 문제가 있는 경우라도, 전체취소를 해서 식당이 손해를 보고 있는 현실이에요
지역에 따른 배달팁 가격이 거리에 따라서 다른 건 이해를 하는데, 그 책정방법에 불만이 있습니다. 어떤 집은 배달팁 자체에서 위치에 따라 자동으로 가격을 매겨주는데, 어떤 집은 메뉴판에 여러 동네를 기입해 놓고 고객이 고르는 방식으로 운영하고 있습니다. 이 부분을 좀 통일해 줬으면 합니다. 철저히 둘중 하나의 방법으로 모두 통일이 되어 있었다면 불만 자체를 갖지 않았을 겁니다. 근데 어떤 집은 고객이 신경써야만 하고, 어떤 집은 그냥 팁에 매겨진 가격대로만 하면 되는 방식이니 상당히 헷갈립니다. 고객입장에서 메뉴판에 있는 음식을 중심으로 보는거지, 집 위치가 가게에서 부터 얼마나 차이 나는지를 신경쓰지 않습니다. 가뜩이나 배달료도 올랐는데 거기에 추가금액 까지 생기는 걸 보면 뭔가 더럽게 손해보는 거 같고 불합리하게 느껴집니다. 메뉴판에서 고르는 거 자체가 이를 주목시킵니다. 점주들도 욕먹을 각오하고 깜빡한 고객에게 전화를 해야하니 무섭고 불편하실 겁니다. 좀 개선해